Since the models were run on python without Jupyter notebooks on the HPC in a job queue, we have the results from the model in a csv and the logs from training in .err and .out files

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSeq2SeqLM
from datasets import load_dataset
from torch.utils.data import DataLoader
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import bitsandbytes as bnb
import transformers
from trl import SFTTrainer, SFTConfig
import logging
import csv
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set the TOKENIZERS_PARALLELISM environment variable
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
def check_cuda():
    logger.info(f"CUDA available: {torch.cuda.is_available()}")
    logger.info(f"CUDA device count: {torch.cuda.device_count()}")
    logger.info(f"Current CUDA device: {torch.cuda.current_device()}")
    logger.info(f"CUDA device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")

# Loading Model and Tokenizer

In [ ]:
def load_model_and_tokenizer(model_id, cache_dir):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, cache_dir=cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True, cache_dir=cache_dir)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"  # Set padding side to right
    return model, tokenizer

# Cleaning the data into a form that the model can consume

In [ ]:
def load_and_clean_dataset(dataset_name, cache_dir):
    dataset = load_dataset(dataset_name, "en", cache_dir=cache_dir, trust_remote_code=True)
    def remove_extra_columns(example):
        example["entities"] = [entity["surfaceform"] for entity in example["entities"]]
        example["relations"] = [
            {
                "subject": example["entities"][relation["subject"]],
                "predicate": relation["predicate"],
                "object": example["entities"][relation["object"]],
            }
            for relation in example["relations"]
        ]
        return example
    return dataset.map(remove_extra_columns)

# Adding the input to our models into the dataset

In [ ]:
def preprocess_function(data_point):
    """
    Convert entities and relations to the expected output text format.
    """
    query = data_point['text']
    entities = ", ".join([f'"{entity}"' for entity in data_point['entities']])
    relations = "\n".join([f'"{relations}"' for relations in data_point['relations']])

    text = f"Given the following text, identify and extract all entities and their relations. Query: {query}\n Entities: [{entities}]\nRelations:\n{relations}"
    return text

# Finding all the layers that LORA can be applied to

In [ ]:
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

# Running Gemma 7b-it

In [ ]:
check_cuda()

model_id = "google/gemma-2b-it"
cache_dir = "SLM/gemma2b"  # Specify an alternative cache directory
dataset_name = "Babelscape/SREDFM"

model, tokenizer = load_model_and_tokenizer(model_id, cache_dir)
dataset = load_and_clean_dataset(dataset_name, cache_dir)

tokenized_datasets = dataset.map(lambda dp: {"model_input": preprocess_function(dp)})
logger.info(tokenized_datasets["test"][0])

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

modules = find_all_linear_names(model)
logger.info(modules)

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

trainable, total = model.get_nb_trainable_parameters()
logger.info(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    dataset_text_field='model_input',
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,  # Increase batch size if memory allows
        gradient_accumulation_steps=8,
        warmup_steps=100,  # Add warmup steps
        max_steps=1000,  # Increase max steps for better training
        learning_rate=2e-4,
        logging_steps=10,  # Log less frequently to reduce overhead
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        fp16=True,  # Use mixed precision training
        dataloader_num_workers=4,  # Optimize data loading
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

new_model = "gemma2b-trained"
trainer.model.save_pretrained(new_model)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
    cache_dir="SLM/gemma2b_trained"  # Specify an alternative cache directory
)
merged_model = PeftModel.from_pretrained(base_model, new_model)
merged_model = merged_model.merge_and_unload()

merged_model.to("cpu")
tokenizer.save_pretrained("gemma2b_trained")
merged_model.save_pretrained("gemma2b_trained", safe_serialization=True)

merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

# Running Gemma7b-it

In [ ]:
check_cuda()

model_id = "google/gemma-7b-it"
cache_dir = "SLM/gemma7b"  # Specify an alternative cache directory
dataset_name = "Babelscape/SREDFM"

model, tokenizer = load_model_and_tokenizer(model_id, cache_dir)
dataset = load_and_clean_dataset(dataset_name, cache_dir)

tokenized_datasets = dataset.map(lambda dp: {"model_input": preprocess_function(dp)})
logger.info(tokenized_datasets["test"][0])

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

modules = find_all_linear_names(model)
logger.info(modules)

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

trainable, total = model.get_nb_trainable_parameters()
logger.info(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    dataset_text_field='model_input',
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,  # Increase batch size if memory allows
        gradient_accumulation_steps=8,
        warmup_steps=100,  # Add warmup steps
        max_steps=1000,  # Increase max steps for better training
        learning_rate=2e-4,
        logging_steps=10,  # Log less frequently to reduce overhead
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        fp16=True,  # Use mixed precision training
        dataloader_num_workers=4,  # Optimize data loading
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

new_model = "gemma7b-trained"
trainer.model.save_pretrained(new_model)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
    cache_dir="SLM/gemma7b_trained"  # Specify an alternative cache directory
)
merged_model = PeftModel.from_pretrained(base_model, new_model)
merged_model = merged_model.merge_and_unload()

merged_model.to("cpu")
tokenizer.save_pretrained("gemma7b_trained")
merged_model.save_pretrained("gemma7b_trained", safe_serialization=True)

merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

# Compiling model results into a CSV

Models have been pushed to hugging-face and can be pulled from there for inference

# Preparing a dataloader for our model

In [ ]:
def get_completion_batch(queries, model, tokenizer, device):
    prompt_template = """
    <start_of_turn>user
    Given the following text, identify and extract all entities and their relations.
    {query}
    <end_of_turn>\n<start_of_turn>model
    """
    # Generate prompts
    prompts = [prompt_template.format(query=query) for query in queries]
    encodeds = tokenizer(
        prompts,
        return_tensors="pt",
        padding="longest",  # Explicitly specify padding
        truncation=True,
        max_length=1024  # Adjust based on your model's max input length
    )
    encodeds = {key: value.to(device) for key, value in encodeds.items()}

    # Mixed precision inference for speedup
    generated_ids = model.generate(
            **encodeds,
            max_new_tokens=500,  # Adjust as per expected output length
            do_sample=False,     # Disable sampling for faster, deterministic results
            pad_token_id=tokenizer.pad_token_id
        )

    outputs = []
    for ids in generated_ids:
        full_output = tokenizer.decode(ids, skip_special_tokens=True)
        if "Entities and Relations:" in full_output:
            relevant_part = full_output.split("Entities and Relations:")[1].strip()
            outputs.append(relevant_part)
        else:
            outputs.append(full_output.strip())  # Fallback to entire output if section is missing
    return outputs

# Loading first 1000 rows of the dataset

In [ ]:
dataset = load_dataset("Babelscape/SREDFM", "en", cache_dir="SLM/datasets", trust_remote_code=True)
test_dataset = dataset['test'].select(range(1000))  # Select the first 1000 rows

In [ ]:
class QueryDataset(torch.utils.data.Dataset):
    def __init__(self, texts, true_entities, true_relations):
        self.texts = texts
        self.true_entities = true_entities
        self.true_relations = true_relations

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            "text": self.texts[idx],
            "true_entities": self.true_entities[idx],
            "true_relations": self.true_relations[idx],
        }

def custom_collate_fn(batch):
    texts = [item["text"] for item in batch]
    true_entities = [item["true_entities"] for item in batch]
    true_relations = [item["true_relations"] for item in batch]
    return texts, true_entities, true_relations

query_dataset = QueryDataset(
    texts=test_dataset['text'],
    true_entities=test_dataset['entities'],
    true_relations=test_dataset['relations']
)
batch_size = 16  # Adjust based on available GPU memory
dataloader = DataLoader(query_dataset, batch_size=batch_size, collate_fn=custom_collate_fn, pin_memory=True)

# Gemma 2b inference

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_path = "Chinmay0701/gemma2b-trained"
tokenizer = AutoTokenizer.from_pretrained(model_path, cache_dir="gemma2b-trained", padding_size = "left")
tokenizer.pad_token = tokenizer.eos_token if tokenizer.pad_token is None else tokenizer.pad_token
tokenizer.padding_side = "left"  # Fix padding side for decoder-only models

model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    torch_dtype=torch.float16 if device == "cuda" else torch.float32, 
    cache_dir="gemma2b-trained"
)
model.to(device)

output_file = "gemma2b_entities_relations.csv"

with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Query", "Predicted Entities", "Predicted Relations", "Output"])  # Write header row

    for batch_idx, (queries, true_entities, true_relations) in enumerate(dataloader):
        print(f"Processing batch {batch_idx + 1}/{len(dataloader)}...")
        
        # Get predictions for the batch
        predicted = get_completion_batch(queries, model, tokenizer, device)

        # Split predicted results into entities and relations
        for query, pred_output, true_ent, true_rel in zip(queries, predicted, true_entities, true_relations):
            if "Entities:" in pred_output and "Relations:" in pred_output:
            # Split the predicted output into entities and relations parts
                parts = pred_output.split("Relations:")
                pred_entities = parts[0].split("Entities:")
                pred_entities = pred_entities[1].replace("Entities:", "").strip()
                pred_relations = parts[1].strip()
            else:
                pred_entities = "NA"
                pred_relations = "NA"
            
            writer.writerow([query,pred_output ,pred_entities, pred_relations])

print(f"Processing complete. Results saved to {output_file}.")

# Gemma 7b Inference

In [ ]:
model_path = "Chinmay0701/gemma7b-trained"  # Path where the model and tokenizer are saved

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, cache_dir="gemma7b-trained", padding_size = "left")
tokenizer.pad_token = tokenizer.eos_token if tokenizer.pad_token is None else tokenizer.pad_token
tokenizer.padding_side = "left" 

output_file = "gemma7b_entities_relations.csv"

with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Query", "Predicted Entities", "Predicted Relations", "Output"])  # Write header row

    for batch_idx, (queries, true_entities, true_relations) in enumerate(dataloader):
        print(f"Processing batch {batch_idx + 1}/{len(dataloader)}...")
        
        # Get predictions for the batch
        predicted = get_completion_batch(queries, model, tokenizer, device)

        # Split predicted results into entities and relations
        for query, pred_output, true_ent, true_rel in zip(queries, predicted, true_entities, true_relations):
            if "Entities:" in pred_output and "Relations:" in pred_output:
            # Split the predicted output into entities and relations parts
                parts = pred_output.split("Relations:")
                pred_entities = parts[0].split("Entities:")
                pred_entities = pred_entities[1].replace("Entities:", "").strip()
                pred_relations = parts[1].strip()
            else:
                pred_entities = "NA"
                pred_relations = "NA"
            
            writer.writerow([query,pred_output ,pred_entities, pred_relations])

print(f"Processing complete. Results saved to {output_file}.")

## Rebel inference

In [ ]:
def extract_triplets(text: str) -> list:
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'subject': subject.strip(), 'predicate': relation.strip(),'object': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'subject': subject.strip(), 'predicate': relation.strip(),'object': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'subject': subject.strip(), 'predicate': relation.strip(),'object': object_.strip()})
    return triplets

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")
gen_kwargs = {
    "max_length": 256,
    "length_penalty": 0,
    "num_beams": 3,
    "num_return_sequences": 3,
}

limit = 1000
test_dataset = load_dataset("Babelscape/SREDFM", "en", split="test", trust_remote_code=True)
test_dataset = test_dataset.map(remove_extra_columns)
entities = test_dataset["entities"][:limit]
relations = test_dataset["relations"][:limit]
texts = test_dataset["text"][:limit]

node_scores = []
relation_scores = []
output_file = "rebel_entities_relations.csv"

# Clear the output file
with open(output_file, "w") as f:
    f.write("")

for text, true_entity, true_relation in zip(texts, entities, relations):
    model_inputs = tokenizer(text, max_length=256, padding=True, truncation=True, 
                             return_tensors = 'pt')
    generated_tokens = model.generate(model_inputs["input_ids"].to(model.device),
                                    attention_mask=model_inputs["attention_mask"].
                                    to(model.device), **gen_kwargs)
    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

    predicted_entities = set()
    predicted_relations = []
    for idx, sentence in enumerate(decoded_preds):
        new_relations = extract_triplets(sentence)
        predicted_relations.extend(new_relations)
        for relation in new_relations:
            predicted_entities.add(relation['subject'])
            predicted_entities.add(relation['object'])

    with open(, "a", encoding='utf-8') as f:
        f.write(str(list(predicted_entities)) + "\n")
        f.write(str(predicted_relations) + "\n")
        f.write("\n#\n")


## Parse output before evaluation

In [ ]:
replacements = [
    ("'Non", "Non"),
    ("[\"\"The First Celestial", "\"\"The First Celestial"),
    ("[\"\"The Silent Serin", "\"\"The Silent Serin"),
    ("[\"\"The Untuned Ventriloquist", "\"\"The Untuned Ventriloquist"),
    ("[\"\"The Cannibal Manifesto in the Dark", "\"\"The Cannibal Manifesto in the Dark")
]

def parse_gemma_output(file_path: str) -> dict:
    with open(file_path, encoding='utf-8') as f:
        data = f.read()

    # Clean ' from the data
    data = re.sub(r"([a-zA-Z0-9])'([a-zA-Z0-9\s-])", r"\1\2", data)
    for replacement in replacements:
        data = data.replace(replacement[0], replacement[1])
    sections = data.split('[')
    all_entities = []
    all_relations = []
    skip = 0
    first_section = True
    for s in sections[1:]:
        if s[:1] == "{" or s[:1].isalnum() or s[:1] in ".=":
            if s[:1] == "{":
                skip += 1
            if not s[:1].isalnum() and not s[:1] in ".=" and s[:1] != "{":
                print(s)
            continue

        # Handle cases where model didnt generate anything
        if skip % 2 != 0:
            raise ValueError("Unbalanced brackets")
        count = skip // 2 - 1
        if first_section:
            count += 1
            first_section = False
        for _ in range(count):
            all_entities.append([])
            all_relations.append([])
        skip = 0

        end_index = s.find(']')
        if end_index != -1:
            entities = s[:end_index].replace("\"\"", "").split(', ')
            relations_as_list = s[end_index + 4:].replace("\"\"", "").split('\n')
            if "}" not in relations_as_list[-1]:
                relations_as_list = relations_as_list[:-1]
            else:
                relations_as_list[-1] = relations_as_list[-1][:relations_as_list[-1].find("}") + 1]
            relations = list(set(relations_as_list))
            try:
                relations = [eval(relation) for relation in relations if relation]
            except SyntaxError as _:
                # print(f"Error parsing the following relations {relations}")
                all_entities.append([])
                all_relations.append([])
                continue

            all_entities.append(entities)
            all_relations.append(relations)
    return {"entities": all_entities, "relations": all_relations}

def parse_rebel_output(file_path: str) -> dict:
    with open(file_path, encoding='utf-8') as f:
        data = f.read()

    sections = data.split('\n#\n')
    all_entities = []
    all_relations = []
    for s in sections[:-1]:
        s = s.strip()
        entities, relations = s.split('\n')
        all_entities.append(eval(entities))
        all_relations.append(eval(relations))

    return {"entities": all_entities, "relations": all_relations}

## Evaluate results

In [ ]:
def compute_graph_metrics(pred_nodes, true_nodes, pred_relations, true_relations):
    # Compute for nodes
    if len(pred_nodes) == 0:
        node_f1 = 0
    else:
        node_precision = len(set(pred_nodes) & set(true_nodes)) / len(set(pred_nodes))
        node_recall = len(set(pred_nodes) & set(true_nodes)) / len(set(true_nodes))
        node_f1 = 2 * node_precision * node_recall / (node_precision + node_recall) if\
                (node_precision + node_recall) > 0 else 0

    # Compute for relationships
    if len(pred_relations) == 0:
        relation_f1 = 0
    else:
        relation_precision = len(set(pred_relations) & set(true_relations)) / len(set(pred_relations))
        relation_recall = len(set(pred_relations) & set(true_relations)) / len(set(true_relations))
        relation_f1 = 2 * relation_precision * relation_recall / (relation_precision + relation_recall) if\
                (relation_precision + relation_recall) > 0 else 0

    return {
        "node_f1": node_f1,
        "relation_f1": relation_f1,
    }

def evaluate(true_nodes: list, predicted_nodes: list, true_relations: dict, predicted_relations: dict) -> dict:
    """Takes in the true and predicted nodes and relations and returns the node and relation scores
    true_nodes: list of true nodes represented as strings
    predicted_nodes: list of predicted nodes represented as strings
    true_relations: dictionary of true relations in format {'subject': str, 'predicate': str, 'object': str}
    predicted_relations: dictionary of predicted relations in format {'subject': str, 'predicate': str, 'object': str}
    """

    if not predicted_nodes and not predicted_relations:
        return {'node_score': 0, 'relations_score': 0}

    true_relations_tuples = [(rel['subject'], rel['predicate'], rel['object'])
                             for rel in true_relations]
    predicted_relations_tuples = [(rel['subject'], rel['predicate'], rel['object'])
                                  for rel in predicted_relations]

    true_relations_text = [f"{rel['subject']} -> {rel['predicate']} -> {rel['object']}"
                            for rel in true_relations]
    predicted_relations_text = [f"{rel['subject']} -> {rel['predicate']} -> {rel['object']}"
                                for rel in predicted_relations]

    metrics = compute_graph_metrics(predicted_nodes, true_nodes,
                                    predicted_relations_tuples, true_relations_tuples)

    # Load embedding model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    true_embeddings = model.encode(true_relations_text)
    pred_embeddings = model.encode(predicted_relations_text)

    if len(pred_embeddings) > 0:
        aligned_similarity = cosine_similarity(true_embeddings, pred_embeddings)
        aligned_similarity = [max(row) for row in aligned_similarity]

        aligned_similarity_mean = sum(aligned_similarity) / len(aligned_similarity)
    else:
        aligned_similarity_mean = 0

    metrics["aligned_similarity"] = aligned_similarity_mean

    f1_weight = 0.3
    relations_score = metrics["relation_f1"] * f1_weight +\
        metrics["aligned_similarity"] * (1-f1_weight)
    return {'node_score': metrics['node_f1'], 'relations_score': relations_score}

to_parse = [
    ("gemma2b_entities_relations.csv", parse_gemma_output, "Gemma 2b"),
    ("gemma7b_entities_relations.csv", parse_gemma_output, "Gemma 7b"),
    ("rebel_entities_relations.csv", parse_rebel_output, "Rebel"),
]

for file_path, parser, name in to_parse:
    node_scores = []
    relations_scores = []
    results = parser(file_path)
    for true_entities, true_relations, pred_entities, pred_relations in zip(
        entities, relations, results['entities'][start:end], results['relations'][start:end]
        ):
        scores = evaluate(true_entities, pred_entities, true_relations, pred_relations)
        node_scores.append(scores['node_score'])
        relations_scores.append(scores['relations_score'])
    print(f"{name} node score: {sum(node_scores) / len(node_scores)}")
    print(f"{name} relations score: {sum(relations_scores) / len(relations_scores)}")
    print(f"{name} node score without missing values: {sum(node_scores) / len([score for score in node_scores if score > 0])}")
    print(f"{name} relations score without missing values: {sum(relations_scores) / len([score for score in relations_scores if score > 0])}")
    print()
